In [71]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


# Temperature

In [72]:
data = pd.read_table('C:/Users/simon/Downloads/climate-data-swissmean_regSwiss_1.4.txt', sep='\t', header=10)

In [73]:
data.head(3)

,time,jan,feb,mar,apr,may,jun,jul,aug,sep,oct,nov,dec,djf,mam,jja,son,winter,summer,year
0,1864,-7.10,-4.52,0.04,2.10,7.43,9.48,12.37,10.95,8.59,3.51,-0.51,-4.47,NaN,3.19,10.93,3.86,NaN,8.49,3.16
1,1865,-3.46,-6.26,-5.91,7.01,10.10,10.98,13.98,11.16,11.89,5.21,1.26,-3.07,-4.73,3.73,12.04,6.12,-2.85,10.85,4.41
2,1866,-1.31,-0.42,-1.00,4.10,4.96,12.02,12.25,9.99,9.69,5.35,-0.20,-0.49,-1.60,2.68,11.42,4.95,0.11,8.83,4.58


In [74]:
df_temp = data[['time', 'year']]
df_temp.tail(2)

,time,year
160,2024,7.17
161,2025,NaN


In [75]:
df_temp.rename(columns={'time': 'year', 'year':'temp'}, inplace=True)

C:\Users\simon\AppData\Local\Temp\ipykernel_23440\2835198872.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp.rename(columns={'time': 'year', 'year':'temp'}, inplace=True)


In [76]:
df_temp.tail(2)

,year,temp
160,2024,7.17
161,2025,NaN


# Pluie

In [77]:
df_pluie = pd.read_csv(f'C:/Users/simon/OneDrive/Bureau/MLPluie/BE.csv', sep=';', header=0)
df_pluie['reference_timestamp'] = pd.to_datetime(df_pluie['reference_timestamp'], format='%d.%m.%Y %H:%M')
df_pluie['year'] = df_pluie['reference_timestamp'].dt.year
df_pluie.drop(columns=['reference_timestamp'], inplace=True)

In [78]:
df_pluie

,station_abbr,rre150y0,year
0,BEP,1067.3,1901
1,BEP,1123.3,1902
2,BEP,1032.5,1903
3,BEP,900.0,1904
4,BEP,1244.1,1905
...,...,...,...
119,BEP,1039.8,2020
120,BEP,1197.9,2021
121,BEP,916.9,2022
122,BEP,1135.2,2023


In [79]:
for i in ['BL', 'BS', 'GE', 'LU', 'NE', 'TG', 'TI', 'VD', 'VS', 'ZH']:
    data = pd.read_csv(f'C:/Users/simon/OneDrive/Bureau/MLPluie/{i}.csv', sep=';', header=0)
    data['reference_timestamp'] = pd.to_datetime(data['reference_timestamp'], format='%d.%m.%Y %H:%M')
    data['year'] = data['reference_timestamp'].dt.year
    data.drop(columns=['reference_timestamp'], inplace=True)
    df_pluie = pd.merge(df_pluie, data, on=['station_abbr', 'rre150y0', 'year'], how='outer', suffixes=('', f'_{i}'))
df_pluie.tail(3)

,station_abbr,rre150y0,year
1186,OPF,1496.8,1995
1187,OPF,1539.4,1999
1188,OPF,1640.6,2001


In [80]:
df_pluie['station_abbr'].unique()

array(['BEP', 'COU', 'CTO', 'ENT', 'ESZ', 'LAB', 'LON', 'LSN', 'MAB',
       'OPF'], dtype=object)

In [81]:
namedict = {
    'BEP':'BE_Bern_urban', 
    'COU':'NE_Chaumont_rural', 
    'CTO':'TI_Lugano_urban', 
    'ENT':'LU_Rigi_rural', 
    'ESZ':'TG_Taenikon_rural', 
    'LAB':'BL_BaselBinningen_suburban', 
    'LON':'GE_Meyrin_suburban', 
    'LSN':'VD_Lausanne_urban', 
    'MAB':'VS_Saxon_rural',
    'OPF':'ZH_Zuerich_urban'
}


#'BS_Basel_urban'
 

In [82]:
df_pluie['Name'] = df_pluie['station_abbr'].map(namedict)

In [83]:
df_pluie.rename(columns={'rre150y0': 'precip'}, inplace=True)

# Climate df

In [84]:
climate_df = pd.merge(df_temp, df_pluie, on='year', how='outer')

In [86]:
climate_df.dropna(inplace=True)

In [88]:
climate_df.drop(columns=['station_abbr'], inplace=True)

In [89]:
climate_df

,year,temp,precip,Name
10,1874,3.79,810.0,VD_Lausanne_urban
11,1875,3.85,1073.1,VD_Lausanne_urban
12,1876,4.23,1095.0,VD_Lausanne_urban
13,1877,4.06,1149.2,VD_Lausanne_urban
14,1878,3.53,1243.7,VD_Lausanne_urban
...,...,...,...,...
1194,2024,7.17,1386.5,BL_BaselBinningen_suburban
1195,2024,7.17,1391.6,GE_Meyrin_suburban
1196,2024,7.17,1206.2,VD_Lausanne_urban
1197,2024,7.17,934.8,VS_Saxon_rural


In [90]:
climate_df.to_csv('C:/Users/simon/OneDrive/Bureau/MLPluie/climate_df.csv', sep=';', index=False)